<a href="https://colab.research.google.com/github/sarayutallady/AI-Based-Cyber-Security-Threats-Prediction-AI-Agent/blob/main/SQL_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Create three tables: Restaurants, Customers, and Orders with appropriate columns.

In [15]:
import sqlite3
import pandas as pd
from IPython.display import display  # 👈 ADD THIS LINE

# Create in-memory database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create tables
cursor.execute('''
CREATE TABLE Restaurants(
    restaurant_id INTEGER PRIMARY KEY,
    name TEXT,
    cuisine TEXT,
    rating REAL,
    city TEXT
)''')

cursor.execute('''
CREATE TABLE Customers(
    customer_id INTEGER PRIMARY KEY,
    name TEXT,
    city TEXT,
    phone TEXT
)''')

cursor.execute('''
CREATE TABLE Orders(
    order_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    restaurant_id INTEGER,
    total_amount REAL,
    order_date TEXT,
    FOREIGN KEY(customer_id) REFERENCES Customers(customer_id),
    FOREIGN KEY(restaurant_id) REFERENCES Restaurants(restaurant_id)
)''')

print("✅ Tables created successfully!\n")

# --- Display all tables created in the database ---
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("📋 Tables present in the database:")
display(tables)

# --- Display each table's schema (columns) ---
for table in tables['name']:
    print(f"\n📊 Structure of '{table}' table:")
    schema = pd.read_sql_query(f"PRAGMA table_info({table});", conn)
    display(schema)


✅ Tables created successfully!

📋 Tables present in the database:


,name
0,Restaurants
1,Customers
2,Orders



📊 Structure of 'Restaurants' table:


,cid,name,type,notnull,dflt_value,pk
0,0,restaurant_id,INTEGER,0,None,1
1,1,name,TEXT,0,None,0
2,2,cuisine,TEXT,0,None,0
3,3,rating,REAL,0,None,0
4,4,city,TEXT,0,None,0



📊 Structure of 'Customers' table:


,cid,name,type,notnull,dflt_value,pk
0,0,customer_id,INTEGER,0,None,1
1,1,name,TEXT,0,None,0
2,2,city,TEXT,0,None,0
3,3,phone,TEXT,0,None,0



📊 Structure of 'Orders' table:


,cid,name,type,notnull,dflt_value,pk
0,0,order_id,INTEGER,0,None,1
1,1,customer_id,INTEGER,0,None,0
2,2,restaurant_id,INTEGER,0,None,0
3,3,total_amount,REAL,0,None,0
4,4,order_date,TEXT,0,None,0


Add at least 5 restaurants with different cuisines and cities.

In [16]:
cursor.executemany('INSERT INTO Restaurants VALUES (?, ?, ?, ?, ?)', [
    (1,'Tandoori Tales','Indian',4.6,'Delhi'),
    (2,'Pasta Palace','Italian',4.3,'Mumbai'),
    (3,'Sushi Sensei','Japanese',4.9,'Bangalore'),
    (4,'Burger Bay','American',4.1,'Hyderabad'),
    (5,'Green Bowl','Vegan',4.2,'Pune')
])
conn.commit()
pd.read_sql("SELECT * FROM Restaurants", conn)


,restaurant_id,name,cuisine,rating,city
0,1,Tandoori Tales,Indian,4.6,Delhi
1,2,Pasta Palace,Italian,4.3,Mumbai
2,3,Sushi Sensei,Japanese,4.9,Bangalore
3,4,Burger Bay,American,4.1,Hyderabad
4,5,Green Bowl,Vegan,4.2,Pune


Insert 5 customers living in different cities.

In [17]:
cursor.executemany('INSERT INTO Customers VALUES (?, ?, ?, ?)', [
    (1,'Aarav Mehta','Delhi','9991112222'),
    (2,'Priya Nair','Mumbai','8882223333'),
    (3,'Rohan Das','Bangalore','7773334444'),
    (4,'Megha Sharma','Hyderabad','6664445555'),
    (5,'Kunal Singh','Pune','9998887777')
])
conn.commit()
pd.read_sql("SELECT * FROM Customers", conn)


,customer_id,name,city,phone
0,1,Aarav Mehta,Delhi,9991112222
1,2,Priya Nair,Mumbai,8882223333
2,3,Rohan Das,Bangalore,7773334444
3,4,Megha Sharma,Hyderabad,6664445555
4,5,Kunal Singh,Pune,9998887777


Record each customer ordering from different restaurants with amount and date.

In [18]:
cursor.executemany('INSERT INTO Orders VALUES (?, ?, ?, ?, ?)', [
    (1,1,1,450.0,'2025-11-01'),
    (2,2,2,300.0,'2025-11-02'),
    (3,3,3,550.0,'2025-11-03'),
    (4,4,4,250.0,'2025-11-04'),
    (5,5,5,700.0,'2025-11-05')
])
conn.commit()
pd.read_sql("SELECT * FROM Orders", conn)



,order_id,customer_id,restaurant_id,total_amount,order_date
0,1,1,1,450.0,2025-11-01
1,2,2,2,300.0,2025-11-02
2,3,3,3,550.0,2025-11-03
3,4,4,4,250.0,2025-11-04
4,5,5,5,700.0,2025-11-05


Retrieve All Orders Above ₹400

In [19]:
pd.read_sql('SELECT * FROM Orders WHERE total_amount > 400', conn)



,order_id,customer_id,restaurant_id,total_amount,order_date
0,1,1,1,450.0,2025-11-01
1,3,3,3,550.0,2025-11-03
2,5,5,5,700.0,2025-11-05


Show order details with restaurant names and cuisines.

In [20]:
q = '''
SELECT o.order_id, r.name AS restaurant, r.cuisine, o.total_amount
FROM Orders o
JOIN Restaurants r ON o.restaurant_id = r.restaurant_id
ORDER BY o.total_amount DESC
'''
pd.read_sql(q, conn)


,order_id,restaurant,cuisine,total_amount
0,5,Green Bowl,Vegan,700.0
1,3,Sushi Sensei,Japanese,550.0
2,1,Tandoori Tales,Indian,450.0
3,2,Pasta Palace,Italian,300.0
4,4,Burger Bay,American,250.0


Count total orders per restaurant (0 if none).

In [21]:
q = '''
SELECT r.name AS restaurant, COUNT(o.order_id) AS total_orders
FROM Restaurants r
LEFT JOIN Orders o ON r.restaurant_id = o.restaurant_id
GROUP BY r.name
ORDER BY total_orders DESC
'''
pd.read_sql(q, conn)


,restaurant,total_orders
0,Tandoori Tales,1
1,Sushi Sensei,1
2,Pasta Palace,1
3,Green Bowl,1
4,Burger Bay,1


Calculate each city’s total revenue from orders.

In [22]:
q = '''
SELECT r.city, SUM(o.total_amount) AS revenue
FROM Orders o
JOIN Restaurants r ON o.restaurant_id = r.restaurant_id
GROUP BY r.city
ORDER BY revenue DESC
'''
pd.read_sql(q, conn)


,city,revenue
0,Pune,700.0
1,Bangalore,550.0
2,Delhi,450.0
3,Mumbai,300.0
4,Hyderabad,250.0


Customers whose total spend exceeds average order value.

In [23]:
q = '''
SELECT c.name AS customer, SUM(o.total_amount) AS spent
FROM Orders o
JOIN Customers c ON o.customer_id = c.customer_id
GROUP BY c.name
HAVING spent > (SELECT AVG(total_amount) FROM Orders)
ORDER BY spent DESC
'''
pd.read_sql(q, conn)


,customer,spent
0,Kunal Singh,700.0
1,Rohan Das,550.0


Fetch latest 3 orders with customer and restaurant details.

In [24]:
q = '''
SELECT o.order_id, o.order_date, c.name AS customer,
       r.name AS restaurant, o.total_amount
FROM Orders o
JOIN Customers c ON o.customer_id = c.customer_id
JOIN Restaurants r ON o.restaurant_id = r.restaurant_id
ORDER BY o.order_date DESC
LIMIT 3
'''
pd.read_sql(q, conn)


,order_id,order_date,customer,restaurant,total_amount
0,5,2025-11-05,Kunal Singh,Green Bowl,700.0
1,4,2025-11-04,Megha Sharma,Burger Bay,250.0
2,3,2025-11-03,Rohan Das,Sushi Sensei,550.0


Increase rating for restaurants with avg order > ₹450 and remove low value orders (<₹300).

In [25]:
cursor.execute('''
UPDATE Restaurants
SET rating = rating + 0.1
WHERE restaurant_id IN (
    SELECT restaurant_id FROM Orders
    GROUP BY restaurant_id HAVING AVG(total_amount) > 450
)''')

cursor.execute('DELETE FROM Orders WHERE total_amount < 300')
conn.commit()

print("✅ Updated ratings & removed low value orders")
pd.read_sql("SELECT * FROM Restaurants", conn)


✅ Updated ratings & removed low value orders


,restaurant_id,name,cuisine,rating,city
0,1,Tandoori Tales,Indian,4.6,Delhi
1,2,Pasta Palace,Italian,4.3,Mumbai
2,3,Sushi Sensei,Japanese,5.0,Bangalore
3,4,Burger Bay,American,4.1,Hyderabad
4,5,Green Bowl,Vegan,4.3,Pune
